In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# 데이터셋 불러오기

In [ ]:
users = pd.read_csv('users.csv').reset_index(drop=True)
ratings = pd.read_csv('ratings.csv').reset_index(drop=True)
movies = pd.read_csv('movies.csv').reset_index(drop=True)

In [ ]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# 943개의 사용자정보, 100,000개의 평가정보, 1682개의 영화정보
print(users.shape)
print(ratings.shape)
print(movies.shape)

(943, 5)
(100000, 4)
(1682, 24)


In [ ]:
print(users.user_id.nunique())
print(movies.movie_id.nunique())

943
1682


# 데이터 전처리

In [ ]:
# timestamp 제거
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


In [ ]:
# train, test 데이터 분리
x = ratings.copy()
# 기존 데이터에서의 각 user_id별로 train set과 test set의 비율을 동일하게 유지
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=100) # stratify:클래스 분포 비율을 맞춰줌

In [ ]:
# # 자연스러운 표현
# y = ratings['rating']
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=x['user_id'])

In [ ]:
x_test['user_id'].value_counts()

,count
user_id,
405,184
655,171
13,159
450,135
276,129
...,...
309,5
651,5
687,5


In [ ]:
# train 데이터로 Full matrix 구하기
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [ ]:
rating_matrix.iloc[:10,:10]

movie_id,1,2,3,4,5,6,7,8,9,10
user_id,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN
7,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,4.0
8,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN


# 유사도 측정

In [ ]:
# (1) cosine 유사도
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# (2) pearson 유사도 (=상관계수)
user_similarity_pearson = pd.DataFrame(np.corrcoef(matrix_dummy),index=rating_matrix.index, columns=rating_matrix.index)

In [ ]:
user_similarity.iloc[:10,:10]

user_id,1,2,3,4,5,6,7,8,9,10
user_id,,,,,,,,,,
1,1.000000,0.138469,0.039942,0.067367,0.254135,0.375793,0.376862,0.258363,0.065003,0.312205
2,0.138469,1.000000,0.072574,0.121933,0.071134,0.180240,0.076774,0.061574,0.135185,0.119798
3,0.039942,0.072574,1.000000,0.215394,0.028070,0.055804,0.041201,0.069162,0.000000,0.036854
4,0.067367,0.121933,0.215394,1.000000,0.042169,0.039678,0.072127,0.129380,0.072274,0.042426
5,0.254135,0.071134,0.028070,0.042169,1.000000,0.154642,0.278835,0.217214,0.072918,0.152873
6,0.375793,0.180240,0.055804,0.039678,0.154642,1.000000,0.390177,0.164554,0.116019,0.397992
7,0.376862,0.076774,0.041201,0.072127,0.278835,0.390177,1.000000,0.237516,0.103189,0.412857
8,0.258363,0.061574,0.069162,0.129380,0.217214,0.164554,0.237516,1.000000,0.024444,0.167409
9,0.065003,0.135185,0.000000,0.072274,0.072918,0.116019,0.103189,0.024444,1.000000,0.149795


# score 계산 (RMSE. 평점예측)

In [ ]:
# 정확도(RMSE)를 계산하는 함수

def RMSE(y_true, y_pred):
    return round(np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2)),2)

def score(model, neighbor_size=None):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    if neighbor_size is not None:
        # neighbor_size가 주어졌을 때
        y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    else:
        # neighbor_size가 필요 없을 때
        y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])

    y_true = np.array(x_test['rating'])

    return RMSE(y_true, y_pred)


# 평점 예측 모델 생성
- user_id 하나와 movie_id 하나를 넘겨받아 해당 사용자의 해당 영화에 대한 평점 예측치 반환

## Ver1 : best-seller

In [ ]:
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
train_mean

,rating
movie_id,
1,3.856716
2,3.208333
3,3.218750
4,3.567742
5,3.285714
...,...
1678,1.000000
1679,3.000000
1680,2.000000


In [ ]:
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]  # 영화별 평균평점
    except:
        rating = 3.0 # 임의값
    return rating



In [ ]:
#[참고] 함수 수행 예시 확인
id_pairs = zip(x_test['user_id'], x_test['movie_id'])
y_pred = np.array([best_seller(user, movie) for (user, movie) in id_pairs])  # x_test 행 수만큼 반복문이 수행됨
y_true = np.array(x_test['rating'])

In [ ]:
best_seller_score = score(best_seller)

## Ver2 : gender별 group화

In [ ]:
# Full matrix를 사용자 데이터와 merge
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id')

# gender별 평점 평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
g_mean

movie_id  sex
1         F      3.758621
          M      3.891129
2         F      3.384615
          M      3.180723
3         F      2.909091
                   ...   
1678      M      1.000000
1679      M      3.000000
1680      M      2.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3039, dtype: float64

In [ ]:
# gender별 평균을 예측치로 돌려주는 함수
def cf_gender(user_id, movie_id):  # x_test 한 행
    try:# movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            # gender_rating = 3.0
            gender_rating = g_mean.loc[movie_id][gender]
        else:
            gender_rating = 3.0
    except:
        gender_rating = 3.0
    return gender_rating


In [ ]:
#[참고] 함수 수행 예시 확인
# id_pairs = zip(x_test['user_id'], x_test['movie_id'])
# t = np.array([cf_gender(user, movie) for (user, movie) in id_pairs])  # x_test 행 수만큼 반복문이 수행됨
# RMSE = round(np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2)),2)

In [ ]:
cf_gender_score = score(cf_gender)

## Ver3. 기본 CF 알고리즘
- 이웃을 전체 사용자로 설정

In [ ]:
# [참고] 함수 수행 예시 확인
# movie_ratings = rating_matrix[1]
# none_rating_idx = movie_ratings[movie_ratings.isnull()].index
# movie_ratings = movie_ratings.dropna()

# sim_scores = user_similarity[1]
# sim_scores = sim_scores.drop(none_rating_idx)

# print(movie_ratings)
# print(sim_scores)

# np.dot(sim_scores, movie_ratings)

In [ ]:

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수,
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating


In [ ]:

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수,
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple_pearson(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity_pearson[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating


In [ ]:
# 정확도 계산
cf_simple_score = score(CF_simple)

In [ ]:
cf_simple_score_pearson = score(CF_simple_pearson)

## Ver4. 이웃을 고려한(KNN) CF

In [ ]:
# Neighbor size를 정해서 예측치를 계산하는 함수
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
##### (1) simple
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(none_rating_idx)
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우
        if neighbor_size == 0:
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
##### (3) Neighbor size가 지정된 경우
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 영화 rating을 neighbor size만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating



In [ ]:
# 정확도 계산
cf_knn_score  = score(cf_knn, neighbor_size=30)

### 최적의 neighbor size 구하기

In [ ]:
# train set으로 full matrix와 cosine similarity 구하기
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0300
Neighbor size = 20 : RMSE = 1.0200
Neighbor size = 30 : RMSE = 1.0100
Neighbor size = 40 : RMSE = 1.0100
Neighbor size = 50 : RMSE = 1.0100
Neighbor size = 60 : RMSE = 1.0100


## Ver5. 사용자의 평가경향을 고려한 CF

In [ ]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis=1) # 사용자별 평균 점수
rating_bias = (rating_matrix.T - rating_mean).T # 각 영화에 대한 사용자의 점수 - 사용자의 평균 점수

In [ ]:
rating_bias

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.343137,-0.656863,0.343137,-0.656863,-0.656863,1.343137,0.343137,-2.656863,1.343137,-0.656863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.260870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.739130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.129771,0.129771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.837838,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,-1.537500,NaN,NaN,0.462500,1.462500,-0.537500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# print(
#       rating_matrix.T.shape,
#       rating_mean.shape
#       )

(1650, 943) (943,)


In [ ]:
# # [참고]테스트
# rating_matrix_mini=rating_matrix.iloc[:10,:10]
# rating_mean_mini=rating_matrix_mini.mean(axis=1)
# rating_matrix_mini.T

user_id,1,2,3,4,5,6,7,8,9,10
movie_id,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
7,4.0,NaN,NaN,NaN,NaN,2.0,NaN,3.0,4.0,4.0
8,1.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
9,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0


In [ ]:


def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기ㅌㄹ
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction



In [ ]:
cf_knn_bias_score = score(CF_knn_bias, 30)


## Ver 6. 신뢰도 가중 방법

In [ ]:
# 사용자별 공통 평가 수 계산
rating_binary1 = np.array((rating_matrix > 0).astype(float)) #(943,1650)
rating_binary2 = rating_binary1.T # (1650,943)
counts = np.dot(rating_binary1, rating_binary2) # (943,943)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)


In [ ]:
rating_binary2.shape

(1650, 943)

In [ ]:

def CF_knn_bias_sig(user_id, movie_id, neighbor_size):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기
        common_counts = counts[user_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

SIG_LEVEL = 3 # 최소 신뢰도(공통 평가 영화 수)
MIN_RATINGS = 2  # 최소 사용자 수(현재 영화를 평가한 사용자 수)
cf_knn_bias_sig_score = score(CF_knn_bias_sig, 30)

## Ver 6. ICBF

In [ ]:
# 아이템 간의 유사도를 계산해야 하므로 원래 사용했던 rating_matrix를 전치
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)

# 아이템 간의 코사인 유사도 저장
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수,
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating



In [ ]:
# 정확도 계산
cf_ibcf_score = score(CF_IBCF)

# Score 비교

In [ ]:
print(f"""
best_seller_score:{best_seller_score}\n
cf_gender_score:{cf_gender_score}\n
cf_simple_score:{cf_simple_score}\n
cf_simple_score_pearson:{cf_simple_score_pearson}\n
cf_knn_score:{cf_knn_score}\n
cf_knn_bias_score:{cf_knn_bias_score}\n
cf_knn_bias_sig_score:{cf_knn_bias_sig_score}\n
cf_ibcf_score:{cf_ibcf_score}
"""
)


best_seller_score:1.03

cf_gender_score:1.04

cf_simple_score:1.02

cf_simple_score_pearson:1.02

cf_knn_score:1.01

cf_knn_bias_score:0.95

cf_knn_bias_sig_score:0.95

cf_ibcf_score:1.02



# 아이템 추천
- 예측된 평점을 정렬하여 상위 n_items 추천

In [ ]:
def recommender(user, n_items=10, neighbor_size=20, prediction_method=''):
    """
    사용자의 선호도를 바탕으로 추천 아이템을 계산하는 함수.

    Parameters:
    user (str): 추천을 받을 사용자 ID
    n_items (int): 추천할 아이템 수 (기본값: 10)
    neighbor_size (int): 이웃의 수 (기본값: 20)
    prediction_method (str): 예상 평점 계산에 사용할 함수 이름 (기본값: 'cf_knn')

    Returns:
    recommended_items (pd.Series): 추천된 아이템 리스트
    """

    # 함수 이름을 기반으로 해당 함수 객체를 가져옴
    try:
        prediction_function = globals()[prediction_method]
    except KeyError:
        raise ValueError(f"Unknown prediction method: {prediction_method}")

    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index  # 이미 평가한 아이템 확인
    items = rating_matrix.loc[user].drop(rated_index)

    # 동적으로 함수 호출
    for item in items.index:
        predictions.append(prediction_function(user, item, neighbor_size))  # 선택된 함수로 예상 평점 계산

    # 예측된 평점을 정렬하여 상위 n_items 추천
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]  # 예상 평점이 가장 높은 아이템 선택

    # 추천된 아이템의 제목 반환
    recommended_items = movies.loc[recommendations.index]['title']

    return recommended_items


In [ ]:
recommender(user=2, n_items=5, neighbor_size=30,prediction_method='CF_knn_bias')

,title
movie_id,
851,"Bloody Child, The (1996)"
1467,"Cure, The (1995)"
1500,Prisoner of the Mountains (Kavkazsky Plennik) ...
1293,Ayn Rand: A Sense of Life (1997)
1449,Golden Earrings (1947)
